Imports

In [ ]:
from pathlib import Path
from utils import new_graph

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"


Criação do grafo

In [ ]:
file = Path.cwd() / "data" / "Email-Enron.txt"

g = new_graph(file)

#### Métricas básicas

Print do grafo

In [ ]:
g.print_graph()

Densidade e Grau médio

<!-- TODO: adicionar formulas de densidade e grau -->

In [ ]:
def density(graph):
  return 2 * graph.E / (graph.V * (graph.V - 1))

def medium_degree(graph):
  sum_degree = sum([graph.degree(i) for i in range(graph.V)])
  return sum_degree / graph.V
  

In [ ]:
print(f"Densidade:\n\t{density(g)}\n")
print(f"Grau médio:\n\t{medium_degree(g)}")


In [ ]:
degrees = [g.degree(i) for i in range(g.V)]

### Histograma

Matplotlib

In [ ]:
# plt.hist(degrees, bins=max(degrees), color='blue', edgecolor='black')

# plt.show()

Plotly

In [ ]:
# df = pd.DataFrame({"degree": degrees})
# fig = px.histogram(df, nbins=max(degrees), x="degree", title="Degree Distribution", labels={"degree": "Degree"}, log_y=True)
# fig.show()

### Distribuição de grau \(p_k\)

Nesta etapa, calculamos a distribuição empírica de graus do grafo.
Para cada valor de grau \(k\), definimos:
$$
p_k \;=\; \frac{N_k}{N}

$$

onde:
- $N$ é o número total de vértices;
- $N_k$ é o número de vértices com grau \(k\);
- $p_k$ é a fração (probabilidade empírica) de vértices com grau \(k\).

Observação importante:
- para construir a distribuição corretamente, usamos os valores **únicos** de \(k\);
- nesse caso, a soma das probabilidades deve ser aproximadamente 1:
$$\sum_k p_k \approx 1$$


In [ ]:
def nk(graph, k):
  count = 0
  for i in range(graph.V):
    if graph.degree(i) == k:
      count += 1
  return count

def n(graph):
  adj = graph.adj
  return len(adj)

def pk(graph, k):
  return nk(graph, k) / n(graph)

In [ ]:
unique_k = sorted(set(degrees))
pk_list = [pk(g, k) for k in unique_k]

### Visualização da lei de potência

Nesta etapa comparamos duas visualizações da distribuição de grau:
- gráfico em escala normal (`log_x=False`, `log_y=False`): curva como \($p_k \sim k^{-\gamma}$\);
- gráfico em escala log-log (`log_x=True`, `log_y=True`): tendência aproximadamente linear.

A escala log-log ajuda a observar a tendência aproximadamente linear esperada em distribuições do tipo potência.

In [ ]:
df = pd.DataFrame({
    "degree": unique_k,
    "probability": pk_list,
})

In [ ]:
fig = px.scatter(
    df,
    x="degree",
    y="probability",
    title="Degree Distribution (p_k vs k)",
    labels={"degree": "Degree (k)", "probability": "P(k)"}
)
fig.update_traces(mode="markers")
fig.show()

In [ ]:
fig = px.scatter(
    df,
    x="degree",
    y="probability",
    log_x=True,
    log_y=True,
    title="Degree Distribution (p_k vs k)",
    labels={"degree": "Degree (k)", "probability": "P(k)"}
)
fig.update_traces(mode="markers")
fig.show()

### 6. Estimativa de \($\gamma$\)

Para lei de potência, usamos:
$$
p_k \sim k^{-\gamma}
$$
Aplicando log em ambos os lados:
$$
\log p_k = -\gamma \log k + C
$$
Então, no gráfico log-log, o coeficiente angular da reta é \(-$\gamma$\).


In [ ]:
fit_df = df[(df["degree"] > 0) & (df["probability"] > 0)].copy()

log_k = np.log(fit_df["degree"].to_numpy())
log_pk = np.log(fit_df["probability"].to_numpy())

slope, intercept = np.polyfit(log_k, log_pk, 1)
gamma = -slope

print(f"gamma estimado: {gamma:.4f}")

In [ ]:
fit_df["fit_log_pk"] = intercept + slope * np.log(fit_df["degree"])
fit_df["fit_probability"] = np.exp(fit_df["fit_log_pk"])

fig = px.scatter(
    fit_df,
    x="degree",
    y="probability",
    log_x=True,
    log_y=True,
    title=f"Degree Distribution (log-log) | gamma ~ {gamma:.3f}",
    labels={"degree": "Degree (k)", "probability": "P(k)"}
)
fig.update_traces(mode="markers", name="dados")
fig.add_scatter(
    x=fit_df["degree"],
    y=fit_df["fit_probability"],
    mode="lines",
    name="ajuste linear em log-log"
)
fig.show()